In [1]:
import pygui as pgui
pgui.create_interface()


# Example usage:

        # F =>  String: Faction                  'Alloyn'              F=Alloyn
        # D =>  String: Deckname                 'Insane'              D~Forge
        # FB => String: Forgeborn               'Ironbeard'            FB=Ironbeard
        # C  => List:   Cardnames               'Digitize'             C~Digitize
        # A  => List:   Forgeborn - Ability     'Army Commander'       A~'Army Commander'
        # K  => Dict:   Composition             'Robot'                K:Robot > 3

Text(value='tmind', description='Username:')

Combobox(value='', description='Databases:', options=('config', 'flavasava', 'ironbrice', 'kckid', 'maru', 'na…

ToggleButtons(description='Reload:', options=('Decks', 'Fusions', 'Entities', 'Forgeborns'), tooltips=('Decks …

Button(description='Load Decks', style=ButtonStyle())

ToggleButtons(button_style='info', description='Faction:', options=('Alloyin', 'Nekrium', 'Tempys', 'Uterra'),…

...

ToggleButtons(button_style='info', description='Faction:', options=('Alloyin', 'Nekrium', 'Tempys', 'Uterra'),…

Dropdown(options=('The Square Proposing Sentry Angels', 'The Raiders of Leading Capitals', 'The Common Wrench …

Button(description='Show Graph', style=ButtonStyle())

Output()

INFO:root:on_username_change started
INFO:root:on_username_change finished, took 0.011217117309570312 seconds
